In [ ]:
!nvidia-smi

Mon Feb 22 15:15:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

neutralize optuned resnet  
neutralize optuned tf  
neutralize emb

In [ ]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

In [ ]:
!pip install -q kaggle
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/Colab\ Notebooks/kaggle.json /root/.kaggle/
# !kaggle competitions download -c jane-street-market-prediction
# !unzip -n *.zip -d train

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!kaggle datasets download -d tzl0000/neutralize-dataset
!unzip -n neutralize-dataset.zip -d neutralize-dataset
!kaggle datasets download -d tzl0000/pickled
!unzip -n pickled.zip -d pickled
!kaggle datasets download -d tzl0000/keras-nn-with-features-neutralization-optuned
!unzip -n keras-nn-with-features-neutralization-optuned.zip -d keras-nn-with-features-neutralization-optuned
!kaggle datasets download -d tzl0000/pytorchresnetstartertrainingoptuned
!unzip -n pytorchresnetstartertrainingoptuned.zip -d pytorchresnetstartertrainingoptuned
!kaggle datasets download -d tzl0000/jane-embnn5-auc-400-400-400n
!unzip -n jane-embnn5-auc-400-400-400n.zip -d jane-embnn5-auc-400-400-400n


neutralize-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  neutralize-dataset.zip
pickled.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pickled.zip
keras-nn-with-features-neutralization-optuned.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  keras-nn-with-features-neutralization-optuned.zip
pytorchresnetstartertrainingoptuned.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pytorchresnetstartertrainingoptuned.zip
jane-embnn5-auc-400-400-400n.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  jane-embnn5-auc-400-400-400n.zip


In [ ]:
!kaggle datasets download -d  takeshikobayashi/nn-with-features-neutralization-ver12
!unzip -n nn-with-features-neutralization-ver12 -d nn-with-features-neutralization-ver12

!kaggle datasets download -d  takeshikobayashi/keras-nn-with-features-neutralization-ver18
!unzip -n keras-nn-with-features-neutralization-ver18 -d keras-nn-with-features-neutralization-ver18


nn-with-features-neutralization-ver12.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  nn-with-features-neutralization-ver12.zip
keras-nn-with-features-neutralization-ver18.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  keras-nn-with-features-neutralization-ver18.zip


In [ ]:
!kaggle datasets download -d tzl0000/pytorch-resnet-startertraining
!unzip -n pytorch-resnet-startertraining -d pytorch-resnet-startertraining

pytorch-resnet-startertraining.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pytorch-resnet-startertraining.zip


In [ ]:
!kaggle competitions download -c jane-street-market-prediction


competition.cpython-37m-x86_64-linux-gnu.so: Skipping, found more recently modified local copy (use --force to force download)
__init__.py: Skipping, found more recently modified local copy (use --force to force download)
example_sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
example_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
features.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!pip install optuna -q
import optuna
import pandas as pd

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from random import choices


SEED = 1111
inference = False
cv = False

tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm

def build_neutralizer(train, features, proportion, return_neut=False):
    """
    Builds neutralzied features, then trains a linear model to predict neutralized features from original
    features and return the coeffs of that model.
    """
    neutralizer = {}
    neutralized_features = np.zeros((train.shape[0], len(features)))
    target = train[['resp', 'bias']].values
    for i, f in enumerate(features):
        # obtain corrected feature
        feature = train[f].values.reshape(-1, 1)
        coeffs = np.linalg.lstsq(target, feature)[0]
        neutralized_features[:, i] = (feature - (proportion * target.dot(coeffs))).squeeze()
        
    # train model to predict corrected features
    neutralizer = np.linalg.lstsq(train[features+['bias']].values, neutralized_features)[0]
    
    if return_neut:
        return neutralized_features, neutralizer
    else:
        return neutralizer

def neutralize_array(array, neutralizer):
    neutralized_array = array.dot(neutralizer)
    return neutralized_array

def test_neutralization():
    dummy_train = train.loc[:100000, :]
    proportion = 1.0
    neutralized_features, neutralizer = build_neutralizer(dummy_train, features, proportion, True)
    dummy_neut_train = neutralize_array(dummy_train[features+['bias']].values, neutralizer)
    
#     assert np.array_equal(neutralized_features, dummy_neut_train)
    print(neutralized_features[0, :10], dummy_neut_train[0, :10])
    
    
import pickle
def save_pickle(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj, f)


def load_pickle(path):
    with open(path, mode='rb') as f:
        obj = pickle.load(f)
    return obj

neutralize_array = load_pickle("neutralize-dataset/neutralize_array.pickle")


In [ ]:

train = pd.read_pickle(f'neutralize-dataset/n_train.pickle')
train_raw = pd.read_pickle(f'pickled/train85.pickle')


features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X = train.loc[:, train.columns.str.contains('feature')]
X_raw = train_raw.loc[:, train_raw.columns.str.contains('feature')]
weights = train.loc[:, "weight"]
#y_train = (train.loc[:, 'action'])

y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

# valid = train.loc[(train.date >= 450) & (train.date < 500)].reset_index(drop=True)

# train = train.loc[train.date < 450].reset_index(drop=True)

X = train.loc[:, train.columns.str.contains('feature')]
# X_val = valid.loc[:, valid.columns.str.contains('feature')]

#y_train = (train.loc[:, 'action'])

y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
# y_val = np.stack([(valid[c] > 0).astype('int') for c in resp_cols]).T


In [ ]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import namedtuple
from sklearn.metrics import log_loss, roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import warnings
warnings.filterwarnings ("ignore")

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

DATA_PATH = 'jane-street-market-prediction/'

NFOLDS = 5

TRAIN = False
#CACHE_PATH = '/content/jane-embnn5-auc-400-400-400n/Jane_EmbNN5_auc_400_400_400.pth'
CACHE_PATH = '/content/pytorch-resnet-startertraining'


def save_pickle(dic, save_path):
    with open(save_path, 'wb') as f:
    # with gzip.open(save_path, 'wb') as f:
        pickle.dump(dic, f)

def load_pickle(load_path):
    with open(load_path, 'rb') as f:
    # with gzip.open(load_path, 'rb') as f:
        message_dict = pickle.load(f)
    return message_dict

feat_cols = [f'feature_{i}' for i in range(130)]

target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

##### Making features
all_feat_cols = [col for col in feat_cols]
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

##### Model&Data fnc
class Model(nn.Module):
    def __init__(self, dropout_rate, hidden_sizes):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        self.dropout0 = nn.Dropout(0.2)

        self.dense1 = nn.Linear(len(all_feat_cols), hidden_sizes[0])
        self.batch_norm1 = nn.BatchNorm1d(hidden_sizes[0])
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_sizes[0]+len(all_feat_cols), hidden_sizes[1])
        self.batch_norm2 = nn.BatchNorm1d(hidden_sizes[1])
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_sizes[0]+hidden_sizes[1], hidden_sizes[2])
        self.batch_norm3 = nn.BatchNorm1d(hidden_sizes[2])
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_sizes[1]+hidden_sizes[2], hidden_sizes[3])
        self.batch_norm4 = nn.BatchNorm1d(hidden_sizes[3])
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_sizes[2]+hidden_sizes[3], len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()


    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

##optuna後
#dropout_rate = 0.200
#hidden_sizes = [224, 256, 96, 160]
    
#もともと
dropout_rate = 0.2
hidden_sizes = [256,256,256,256]

if True:
    device = torch.device("cuda:0")
    # device = torch.device("cpu")
    model_list = []
    
    ###Foldありのモデル
    for _fold in range(NFOLDS):
        torch.cuda.empty_cache()
        model = Model(dropout_rate, hidden_sizes)
        model.to(device)
        
        model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
        # model.load_state_dict(torch.load(model_weights, map_location=torch.device('cpu')))
        model.load_state_dict(torch.load(model_weights, map_location=torch.device('cuda:0')))
        
        print(f'weight load {_fold} done!')

        model.eval()
        model_list.append(model)



weight load 0 done!
weight load 1 done!
weight load 2 done!
weight load 3 done!
weight load 4 done!


In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
# import tensorflow_addons as tfa

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices


SEED = 1111

np.random.seed(SEED)

# fit
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

##optuna params
#batch_size = 5000
#hidden_units = [272, 304, 512]
#dropout_rates = [0.05, 0.45, 0.1, 0.3]
#label_smoothing = 1e-2
#learning_rate = 1e-3
#epochs = 200

##ver9
batch_size = 4096
hidden_units = [192, 384, 192]
dropout_rates = [0.10143786981358652, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
label_smoothing = 1e-2
learning_rate = 1e-3
epochs = 200


tf.keras.backend.clear_session()
tf.random.set_seed(SEED)


KFOLD = 5
#modelpath = '/content/keras-nn-with-features-neutralization-ver18/keras_nn_fold'
modelpath = '/content/nn-with-features-neutralization-ver12/keras_nn_fold'
modelfile = '.hdf5'

clf = create_mlp(
    len(feat_cols), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

tf_models = []


for fold in range(KFOLD):
        
    modelno = fold
    bbb = modelpath + str(modelno) + modelfile
    print(bbb)
    clf.load_weights(bbb)
    tf_models.append(clf)
        
    print('weight load Done!')    


neutralize_array = load_pickle("neutralize-dataset/neutralize_array.pickle")

#neutralize_array = load_pickle("../input/neutralize-dataset/neutralize_array.pickle")


#clf = create_mlp(
#    len(feat_cols), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
#    )
#clf2 = create_mlp(
#    len(feat_cols), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
#    )
## clf.load_weights("keras-nn-with-features-neutralization-optuned/keras_nn_fold2.hdf5")
#clf.load_weights("keras-nn-with-features-neutralization-optuned/keras_nn_fold1.hdf5")


#clf2.load_weights("keras-nn-with-features-neutralization-optuned/keras_nn_fold4.hdf5")


#neutralize_array = load_pickle("neutralize-dataset/neutralize_array.pickle")

#tf_models = [clf]

/content/nn-with-features-neutralization-ver12/keras_nn_fold0.hdf5
weight load Done!
/content/nn-with-features-neutralization-ver12/keras_nn_fold1.hdf5
weight load Done!
/content/nn-with-features-neutralization-ver12/keras_nn_fold2.hdf5
weight load Done!
/content/nn-with-features-neutralization-ver12/keras_nn_fold3.hdf5
weight load Done!
/content/nn-with-features-neutralization-ver12/keras_nn_fold4.hdf5
weight load Done!


In [ ]:
N_FEAT_TAGS = 29    # No of tags in features.csv
DEVICE = device
N_FEATURES = 130
THREE_HIDDEN_LAYERS = [400, 400, 400]


class FFN (nn.Module):
    
    def __init__(self, inputCount=130, outputCount=5, hiddenLayerCounts=[150, 150, 150], 
                 drop_prob=0.2, nonlin=nn.SiLU (), isOpAct=False):
        
        super(FFN, self).__init__()
        
        self.nonlin     = nonlin
        self.dropout    = nn.Dropout (drop_prob)
        self.batchnorm0 = nn.BatchNorm1d (inputCount)
        self.dense1     = nn.Linear (inputCount, hiddenLayerCounts[0])
        self.batchnorm1 = nn.BatchNorm1d (hiddenLayerCounts[0])
        self.dense2     = nn.Linear(hiddenLayerCounts[0], hiddenLayerCounts[1])
        self.batchnorm2 = nn.BatchNorm1d (hiddenLayerCounts[1])
        self.dense3     = nn.Linear(hiddenLayerCounts[1], hiddenLayerCounts[2])
        self.batchnorm3 = nn.BatchNorm1d (hiddenLayerCounts[2])        
        self.outDense   = None
        if outputCount > 0:
            self.outDense   = nn.Linear (hiddenLayerCounts[-1], outputCount)
        self.outActivtn = None
        if isOpAct:
            if outputCount == 1 or outputCount == 2:
                self.outActivtn = nn.Sigmoid ()
            elif outputCount > 0:
                self.outActivtn = nn.Softmax (dim=-1)
        return

    def forward (self, X):
        
        # X = self.dropout (self.batchnorm0 (X))
        X = self.batchnorm0 (X)
        X = self.dropout (self.nonlin (self.batchnorm1 (self.dense1 (X))))
        X = self.dropout (self.nonlin (self.batchnorm2 (self.dense2 (X))))
        X = self.dropout (self.nonlin (self.batchnorm3 (self.dense3 (X))))
        if self.outDense:
            X = self.outDense (X)
        if self.outActivtn:
            X = self.outActivtn (X)
        return X
    
    
class Emb_NN_Model (nn.Module):
    
    def __init__(self, three_hidden_layers=THREE_HIDDEN_LAYERS, embed_dim=(N_FEAT_TAGS), csv_file='features.csv'):
        
        super (Emb_NN_Model, self).__init__()
        global N_FEAT_TAGS
        N_FEAT_TAGS = 29
        
        # store the features to tags mapping as a datframe tdf, feature_i mapping is in tdf[i, :]
        dtype = {'tag_0' : 'int8'}
        for i in range (1, 29):
            k = 'tag_' + str (i)
            dtype[k] = 'int8'
        t_df = pd.read_csv (csv_file, usecols=range (1,N_FEAT_TAGS+1), dtype=dtype)
        t_df['tag_29'] = np.array ([1] + ([0] * (t_df.shape[0]-1)) ).astype ('int8')
        self.features_tag_matrix = torch.tensor (t_df.to_numpy ())
        N_FEAT_TAGS += 1
        
        # print ('self.features_tag_matrix =', self.features_tag_matrix)
        
        # embeddings for the tags. Each feature is taken a an embedding which is an avg. of its' tag embeddings
        self.embed_dim     = embed_dim
        self.tag_embedding = nn.Embedding (N_FEAT_TAGS+1, embed_dim) # create a special tag if not known tag for any feature
        self.tag_weights   = nn.Linear (N_FEAT_TAGS, 1)
        
        drop_prob          = 0.5
        self.ffn           = FFN (inputCount=(130+embed_dim), outputCount=0, hiddenLayerCounts=[(three_hidden_layers[0]+embed_dim), (three_hidden_layers[1]+embed_dim), (three_hidden_layers[2]+embed_dim)], drop_prob=drop_prob)
        self.outDense      = nn.Linear (three_hidden_layers[2]+embed_dim, 5)
        return
    
    def features2emb (self):
        """
        idx : int feature index 0 to N_FEATURES-1 (129)
        """
        
        all_tag_idxs = torch.LongTensor (np.arange (N_FEAT_TAGS)) #.to (DEVICE)              # (29,)
        tag_bools    = self.features_tag_matrix                                # (130, 29)
        # print ('tag_bools.shape =', tag_bools.size())
        f_emb        = self.tag_embedding (all_tag_idxs).repeat (130, 1, 1)    #;print ('1. f_emb =', f_emb) # (29, 7) * (130, 1, 1) = (130, 29, 7)
        # print ('f_emb.shape =', f_emb.size())
        f_emb        = f_emb * tag_bools[:, :, None]                           #;print ('2. f_emb =', f_emb) # (130, 29, 7) * (130, 29, 1) = (130, 29, 7)
        # print ('f_emb.shape =', f_emb.size())
        
        # Take avg. of all the present tag's embeddings to get the embedding for a feature
        s = torch.sum (tag_bools, dim=1)                                       # (130,)
        # print ('s =', s)              
        f_emb = torch.sum (f_emb, dim=-2) / s[:, None]                         # (130, 7)
        # print ('f_emb =', f_emb)        
        # print ('f_emb.shape =', f_emb.shape)
        
        # take a linear combination of the present tag's embeddings
        # f_emb = f_emb.permute (0, 2, 1)                                        # (130, 7, 29)
        # f_emb = self.tag_weights (f_emb)                      #;print ('3. f_emb =', f_emb)                 # (130, 7, 1)
        # f_emb = torch.squeeze (f_emb, dim=-1)                 #;print ('4. f_emb =', f_emb)                 # (130, 7)
        return f_emb
    
    def forward (self, cat_featrs, features):
        """
        when you call `model (x ,y, z, ...)` then this method is invoked
        """
        
        cat_featrs = None
        features   = features.view (-1, N_FEATURES)
        f_emb      = self.features2emb ()                                #;print ('5. f_emb =', f_emb); print ('6. features =', features) # (130, 7)
        # print ('features.shape =', features.shape, 'f_emb.shape =', f_emb.shape)
        features_2 = torch.matmul (features, f_emb)                      #;print ('7. features =', features) # (1, 130) * (130, 7) = (1, 7)
        # print ('features.shape =', features.shape)
        
        # Concatenate the two features (features + their embeddings)
        features   = torch.hstack ((features, features_2))        
        
        x          = self.ffn (features)                               #;print ('8. x.shape = ', x.shape, 'x =', x)   # (1, 7) -> (1, 7)
        # x        = self.layer_normal (x + features)                  #;print ('9. x.shape = ', x.shape, 'x =', x)   # (1, 7) -> (1, 2)
        out_logits = self.outDense (x)                                 #;print ('10. out_logits.shape = ', out_logits.shape, 'out_logits =', out_logits)        
        # return sigmoid probs
        # out_probs = F.sigmoid (out_logits)
        return out_logits

In [ ]:
embNN_model = Emb_NN_Model ()

try:
    embNN_model.load_state_dict (torch.load ("jane-embnn5-auc-400-400-400n/Jane_EmbNN5_auc_400_400_400.pth"))
except:
    embNN_model.load_state_dict (torch.load ("jane-embnn5-auc-400-400-400n/Jane_EmbNN5_auc_400_400_400.pth", map_location='cpu'))
    
embNN_model = embNN_model.eval ()

In [ ]:
import os
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.model_selection import GroupKFold

In [ ]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    # print('weight: ', weight)
    # print('resp: ', resp)
    # print('action: ', action)
    # print('weight * resp * action: ', weight * resp * action)
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [ ]:
#timeout封じ
#tf_models = tf_models[tf_models[1]:tf_models[4]]
tf_models = [tf_models[1],tf_models[4]]
#model_list = model_list[-2:]
model_list = [model_list[1], model_list[3]]
NFOLDS2 = len(model_list)

tf_models

In [ ]:
batch_size = 4096 * 4

label_smoothing = 1e-2
learning_rate = 1e-3
epochs = 200


def objective(trial):
    resnet_weight = trial.suggest_float("torch_weight", 0, 1)
    tf_weight = trial.suggest_float("tf_weight", 0, 1-resnet_weight)
    #tf_weight = trial.suggest_float("tf_weight", 0, 1 - resnet_weight)
    emb_weight = 1 - (resnet_weight + tf_weight)
    th = trial.suggest_float("threshold", 0.45, 0.55)
    
    print("")
    print(f'trial: {trial._trial_id}\t th: {th:.3f}\t resnet: {resnet_weight:.3f}\t tf: {tf_weight:.3f}\t emb: {emb_weight:.3f}\t')
     
    # cv strategy
    N_SPLITS = 5
    gkf = GroupKFold(n_splits=N_SPLITS)

    auc_score = []
    u_score = []

    for fold, (train_idx, val_idx) in enumerate(gkf.split(train.action.values, groups=train.date.values)):

        # val_idxをtrainに放り込んだ。train_idxはoom
        X_train = X.iloc[val_idx]
        X_raw_train = X_raw.iloc[val_idx]

        x_weight = weights.iloc[val_idx]
        dates = train.date.iloc[val_idx]
        resps = train.resp.iloc[val_idx]

        y_train = y[val_idx]
    
        x_tt = X_train.loc[:, feat_cols].values
        x_tt_r = X_raw_train.loc[:, feat_cols].values

        if np.isnan(x_tt.sum()):
            x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean
        if np.isnan(x_tt_r.sum()):
            x_tt_r = np.nan_to_num(x_tt_r) + np.isnan(x_tt_r) * f_mean        
        cross_41_42_43 = x_tt_r[:, 41] + x_tt_r[:, 42] + x_tt_r[:, 43]
        cross_1_2 = x_tt_r[:, 1] / (x_tt_r[:, 2] + 1e-5)
        # del X_raw_train, X_raw_val, x_tt_r
        # del X_raw_train, x_tt_r

        x_tt_n = np.append(x_tt, [[1]] * x_tt.shape[0], axis=1)
        x_tt_n = neutralize_array(x_tt_n)
        feature_inp = np.concatenate((
            x_tt_n,
            np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
            np.array(cross_1_2).reshape(x_tt.shape[0], 1),
        ), axis=1)
        # torch_pred
        torch_pred = np.zeros((x_tt_n.shape[0], len(target_cols)))
        for model in model_list:
            torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS2
            #torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
        torch_pred = np.median(torch_pred, axis=1)
        
        # tf_pred
        tf_pred = np.median(np.mean([model(x_tt_n, training = False).numpy() for model in tf_models],axis=0), axis=1)
        
        # torch embedding_NN pred
        x_tt_n    = torch.tensor (x_tt_n).float ().view (-1, 130)
        with torch.no_grad():
            temp0 = embNN_model (None, x_tt_n)

        temp1 = torch.sigmoid (temp0).detach ()

        temp2 = temp1.cpu ().numpy ().reshape ((-1, 5))

        embnn_p = np.median (temp2, axis=1)   # not logits, actually sigmoid probabilities
        

        # avg
        pred_pr = torch_pred*resnet_weight + tf_pred*tf_weight + embnn_p*emb_weight
        
        score = roc_auc_score(np.where(resps>0, 1, 0), pred_pr)  # classification score


        pred = np.where(pred_pr >= th, 1, 0).astype(int)
        fil = (x_weight == 0)
        pred[fil] = 0


        valid_score = utility_score_bincount(
            date=dates.values,
            weight=x_weight.values,
            resp=resps.values,
            action=pred
            )
        print(f'FOLD {fold} ROC AUC:\t {score}\t SCORE: {valid_score}\t TIME: {(time.time() - start)/60:.1f}min')

        # deleting excess data to avoid running out of memory
        # del X_train, X_val, y_train, y_val, X_raw_train, X_raw_val
        del X_train, y_train, X_raw_train, 
        

        auc_score.append(score)
        u_score.append(valid_score)

        if (fold >= 1) and (np.mean(u_score) < 2500): # early stop
            break
        # # appending val_idx in case of group time series split
        # val_idx_all.append(val_idx)
        
        # # concatenation of all val_idx for further acessing
        # val_idx = np.concatenate(val_idx_all)

    u_avg = np.mean(u_score)
    
    print(f'model valid score: {u_avg:.4f}\tauc_score: {np.mean(auc_score):.4f}\t', flush=True)
    return u_avg
    # return auc_score

In [ ]:
study = optuna.create_study(storage="sqlite:///opt.db",
                    study_name="score_9",
                    direction="maximize",
                    load_if_exists=True)

[I 2021-02-22 15:17:15,318] Using an existing study with name 'score_9' instead of creating a new one.


In [ ]:
import gc
gc.collect()

2055

In [ ]:
start = time.time()

study.optimize(objective, n_trials=20)
print(study.best_trial)
optuna.visualization.plot_contour(study)


trial: 170	 th: 0.500	 resnet: 0.557	 tf: 0.000	 emb: 0.443	
FOLD 0 ROC AUC:	 0.5390006757044365	 SCORE: 3098.5604574371473	 TIME: 0.2min
FOLD 1 ROC AUC:	 0.5474191254385775	 SCORE: 3413.715339443426	 TIME: 0.3min
FOLD 2 ROC AUC:	 0.541444822220041	 SCORE: 3079.4327726858564	 TIME: 0.5min
FOLD 3 ROC AUC:	 0.5342591845903762	 SCORE: 298.2965685627032	 TIME: 0.6min
model valid score: 2472.5013	auc_score: 0.5405	


[I 2021-02-22 15:17:52,560] Trial 0 finished with value: 2472.5012845322826 and parameters: {'torch_weight': 0.5569601462545118, 'tf_weight': 0.00036046701272512375, 'threshold': 0.5000382654487356}. Best is trial 0 with value: 2472.5012845322826.



trial: 171	 th: 0.483	 resnet: 0.097	 tf: 0.457	 emb: 0.446	
FOLD 0 ROC AUC:	 0.624483905179706	 SCORE: 9375.81516494949	 TIME: 0.8min
FOLD 1 ROC AUC:	 0.6224830464679585	 SCORE: 9922.422041271755	 TIME: 0.9min
FOLD 2 ROC AUC:	 0.6233751353559125	 SCORE: 9516.996477352093	 TIME: 1.0min
FOLD 3 ROC AUC:	 0.6201102532616427	 SCORE: 6804.759026956418	 TIME: 1.2min
FOLD 4 ROC AUC:	 0.5686670248409614	 SCORE: 4949.74678978758	 TIME: 1.3min
model valid score: 8113.9479	auc_score: 0.6118	


[I 2021-02-22 15:18:35,422] Trial 1 finished with value: 8113.947900063467 and parameters: {'torch_weight': 0.09664348012370216, 'tf_weight': 0.4569713593563755, 'threshold': 0.48308436517789716}. Best is trial 1 with value: 8113.947900063467.



trial: 172	 th: 0.529	 resnet: 0.203	 tf: 0.039	 emb: 0.759	
FOLD 0 ROC AUC:	 0.5479198004610566	 SCORE: 633.2496088388316	 TIME: 1.5min
FOLD 1 ROC AUC:	 0.5506268676019499	 SCORE: 762.6517302573279	 TIME: 1.6min
model valid score: 697.9507	auc_score: 0.5493	


...


trial: 188	 th: 0.489	 resnet: 0.384	 tf: 0.615	 emb: 0.001	
FOLD 0 ROC AUC:	 0.6199838614024521	 SCORE: 9484.754975570739	 TIME: 12.1min
FOLD 1 ROC AUC:	 0.6186334984342483	 SCORE: 9943.364201834154	 TIME: 12.3min
FOLD 2 ROC AUC:	 0.619009419073783	 SCORE: 9672.372817952142	 TIME: 12.4min
FOLD 3 ROC AUC:	 0.6149990973762168	 SCORE: 7205.824915117835	 TIME: 12.6min
FOLD 4 ROC AUC:	 0.56797476090021	 SCORE: 5369.429119872041	 TIME: 12.7min
model valid score: 8335.1492	auc_score: 0.6081	


[I 2021-02-22 15:29:57,247] Trial 18 finished with value: 8335.149206069382 and parameters: {'torch_weight': 0.38386224823471843, 'tf_weight': 0.6150594669813364, 'threshold': 0.48924594907575203}. Best is trial 13 with value: 8686.186462753778.



trial: 189	 th: 0.490	 resnet: 0.727	 tf: 0.196	 emb: 0.076	
FOLD 0 ROC AUC:	 0.5881660388471219	 SCORE: 7361.497758963874	 TIME: 12.8min
FOLD 1 ROC AUC:	 0.5919256181072088	 SCORE: 7964.179286983568	 TIME: 13.0min
FOLD 2 ROC AUC:	 0.5889069551532368	 SCORE: 7612.284116032693	 TIME: 13.1min
FOLD 3 ROC AUC:	 0.5828734950632565	 SCORE: 4899.741162999132	 TIME: 13.3min
FOLD 4 ROC AUC:	 0.5569964405459049	 SCORE: 4430.4970903638905	 TIME: 13.4min
model valid score: 6453.6399	auc_score: 0.5818	


[I 2021-02-22 15:30:39,046] Trial 19 finished with value: 6453.639883068632 and parameters: {'torch_weight': 0.7272982506404297, 'tf_weight': 0.19631279830415338, 'threshold': 0.49007020065439905}. Best is trial 13 with value: 8686.186462753778.


FrozenTrial(number=13, values=[8686.186462753778], datetime_start=datetime.datetime(2021, 2, 22, 15, 25, 46, 194264), datetime_complete=datetime.datetime(2021, 2, 22, 15, 26, 28, 23713), params={'tf_weight': 0.692257029795079, 'threshold': 0.4894661826050896, 'torch_weight': 0.05248931578654781}, distributions={'tf_weight': UniformDistribution(high=0.9475106842134522, low=0), 'threshold': UniformDistribution(high=0.55, low=0.45), 'torch_weight': UniformDistribution(high=1, low=0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=183, state=TrialState.COMPLETE, value=None)


In [ ]:
study.optimize(objective, n_trials=20)
print(study.best_trial)
optuna.visualization.plot_contour(study)


trial: 190	 th: 0.509	 resnet: 0.460	 tf: 0.532	 emb: 0.007	
FOLD 0 ROC AUC:	 0.616416990108422	 SCORE: 8479.149762721368	 TIME: 13.6min
FOLD 1 ROC AUC:	 0.6156686270266221	 SCORE: 8912.448248802204	 TIME: 13.7min
FOLD 2 ROC AUC:	 0.6159370115301614	 SCORE: 8859.026080515121	 TIME: 13.8min
FOLD 3 ROC AUC:	 0.6115807241648566	 SCORE: 6701.146591570631	 TIME: 14.0min
FOLD 4 ROC AUC:	 0.5670181008912061	 SCORE: 5086.657031506245	 TIME: 14.1min
model valid score: 7607.6855	auc_score: 0.6053	


[I 2021-02-22 15:31:22,517] Trial 20 finished with value: 7607.6855430231135 and parameters: {'torch_weight': 0.46012293062949294, 'tf_weight': 0.532447685140057, 'threshold': 0.5093537092268045}. Best is trial 13 with value: 8686.186462753778.



trial: 191	 th: 0.485	 resnet: 0.130	 tf: 0.867	 emb: 0.003	
FOLD 0 ROC AUC:	 0.6274061058191972	 SCORE: 9890.117440097118	 TIME: 14.3min
FOLD 1 ROC AUC:	 0.6248825293652728	 SCORE: 10358.182266727003	 TIME: 14.4min
FOLD 2 ROC AUC:	 0.6253007331449645	 SCORE: 10009.979842101651	 TIME: 14.5min
FOLD 3 ROC AUC:	 0.6219589433135326	 SCORE: 7464.257064109015	 TIME: 14.7min
FOLD 4 ROC AUC:	 0.5693747567217669	 SCORE: 5331.632863512403	 TIME: 14.8min
model valid score: 8610.8339	auc_score: 0.6138	


[I 2021-02-22 15:32:04,105] Trial 21 finished with value: 8610.833895309437 and parameters: {'torch_weight': 0.1300619541640185, 'tf_weight': 0.8667324460323736, 'threshold': 0.4846807049546232}. Best is trial 13 with value: 8686.186462753778.



trial: 192	 th: 0.488	 resnet: 0.130	 tf: 0.852	 emb: 0.019	
FOLD 0 ROC AUC:	 0.6273802850057917	 SCORE: 9869.777641873457	 TIME: 15.0min
FOLD 1 ROC AUC:	 0.6248675778771784	 SCORE: 10340.707528405403	 TIME: 15.1min
FOLD 2 ROC AUC:	 0.6252900996243883	 SCORE: 10105.39137713411	 TIME: 15.2min
FOLD 3 ROC AUC:	 0.6219605063165916	 SCORE: 7529.35067083906	 TIME: 15.4min
FOLD 4 ROC AUC:	 0.5693859298820583	 SCORE: 5376.840327290361	 TIME: 15.5min
model valid score: 8644.4135	auc_score: 0.6138	


...


trial: 208	 th: 0.525	 resnet: 0.003	 tf: 0.985	 emb: 0.012	
FOLD 0 ROC AUC:	 0.6296774573004124	 SCORE: 8507.444379696966	 TIME: 26.1min
FOLD 1 ROC AUC:	 0.6267773788231561	 SCORE: 9063.5002770537	 TIME: 26.2min
FOLD 2 ROC AUC:	 0.6272110545966499	 SCORE: 8802.618858086164	 TIME: 26.4min
FOLD 3 ROC AUC:	 0.6240371104311493	 SCORE: 7080.192953704502	 TIME: 26.5min
FOLD 4 ROC AUC:	 0.5695192032025399	 SCORE: 4926.617291224338	 TIME: 26.7min
model valid score: 7676.0748	auc_score: 0.6154	


[I 2021-02-22 15:43:54,609] Trial 38 finished with value: 7676.074751953134 and parameters: {'torch_weight': 0.002721246482785697, 'tf_weight': 0.9847837870344371, 'threshold': 0.5253817668406363}. Best is trial 37 with value: 8750.009215883245.



trial: 209	 th: 0.502	 resnet: 0.255	 tf: 0.328	 emb: 0.418	
FOLD 0 ROC AUC:	 0.6159581075056149	 SCORE: 8326.628479969952	 TIME: 26.8min
FOLD 1 ROC AUC:	 0.6153208660005985	 SCORE: 8829.86083009086	 TIME: 26.9min
FOLD 2 ROC AUC:	 0.6161197302532964	 SCORE: 8829.569970918848	 TIME: 27.1min
FOLD 3 ROC AUC:	 0.6120851153807031	 SCORE: 6718.500160363339	 TIME: 27.2min
FOLD 4 ROC AUC:	 0.5669973370898513	 SCORE: 4993.463733136424	 TIME: 27.3min
model valid score: 7539.6046	auc_score: 0.6053	


[I 2021-02-22 15:44:36,426] Trial 39 finished with value: 7539.604634895884 and parameters: {'torch_weight': 0.25451112744225013, 'tf_weight': 0.3276534370472275, 'threshold': 0.5020763031345538}. Best is trial 37 with value: 8750.009215883245.


FrozenTrial(number=37, values=[8750.009215883245], datetime_start=datetime.datetime(2021, 2, 22, 15, 42, 31, 323943), datetime_complete=datetime.datetime(2021, 2, 22, 15, 43, 12, 880676), params={'tf_weight': 0.9644663302790105, 'threshold': 0.49398700104586946, 'torch_weight': 0.00636465508215267}, distributions={'tf_weight': UniformDistribution(high=0.9936353449178473, low=0), 'threshold': UniformDistribution(high=0.55, low=0.45), 'torch_weight': UniformDistribution(high=1, low=0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=207, state=TrialState.COMPLETE, value=None)


In [ ]:
study.best_params

{'tf_weight': 0.9644663302790105,
 'threshold': 0.49398700104586946,
 'torch_weight': 0.00636465508215267}

In [ ]:
study.optimize(objective, n_trials=10)
print(study.best_trial)
optuna.visualization.plot_contour(study)


trial: 210	 th: 0.514	 resnet: 0.339	 tf: 0.260	 emb: 0.400	
FOLD 0 ROC AUC:	 0.6089249024231105	 SCORE: 6560.631067420775	 TIME: 27.5min
FOLD 1 ROC AUC:	 0.6093841832827576	 SCORE: 6908.051643048044	 TIME: 27.6min
FOLD 2 ROC AUC:	 0.6099368345414327	 SCORE: 7036.391006295709	 TIME: 27.8min
FOLD 3 ROC AUC:	 0.6051180151318509	 SCORE: 5564.4306965834285	 TIME: 27.9min
FOLD 4 ROC AUC:	 0.5649633577541564	 SCORE: 4248.372975665722	 TIME: 28.1min
model valid score: 6063.5755	auc_score: 0.5997	


[I 2021-02-22 15:45:18,863] Trial 40 finished with value: 6063.575477802736 and parameters: {'torch_weight': 0.3393884972901018, 'tf_weight': 0.2603932208307521, 'threshold': 0.514195376245322}. Best is trial 37 with value: 8750.009215883245.



trial: 211	 th: 0.494	 resnet: 0.069	 tf: 0.900	 emb: 0.031	
FOLD 0 ROC AUC:	 0.6285663913339635	 SCORE: 9843.999271327566	 TIME: 28.2min
FOLD 1 ROC AUC:	 0.6258658054143933	 SCORE: 10345.899075775571	 TIME: 28.3min
FOLD 2 ROC AUC:	 0.6262961514132656	 SCORE: 10160.098103366476	 TIME: 28.5min
FOLD 3 ROC AUC:	 0.6230634121038289	 SCORE: 7682.5019483201995	 TIME: 28.6min
FOLD 4 ROC AUC:	 0.5694954348099881	 SCORE: 5562.538620485638	 TIME: 28.8min
model valid score: 8719.0074	auc_score: 0.6147	


[I 2021-02-22 15:46:00,690] Trial 41 finished with value: 8719.007403855092 and parameters: {'torch_weight': 0.06869862514495927, 'tf_weight': 0.8998346982231966, 'threshold': 0.4937910605842569}. Best is trial 37 with value: 8750.009215883245.



trial: 212	 th: 0.496	 resnet: 0.087	 tf: 0.910	 emb: 0.003	
FOLD 0 ROC AUC:	 0.6282589380003986	 SCORE: 9794.914186692848	 TIME: 28.9min
FOLD 1 ROC AUC:	 0.6255967159098801	 SCORE: 10354.219041534205	 TIME: 29.0min
FOLD 2 ROC AUC:	 0.6260180636342474	 SCORE: 10063.414734970182	 TIME: 29.2min
FOLD 3 ROC AUC:	 0.6227391681997949	 SCORE: 7669.663461546085	 TIME: 29.3min
FOLD 4 ROC AUC:	 0.5694519079821985	 SCORE: 5541.900396204646	 TIME: 29.4min
model valid score: 8684.8224	auc_score: 0.6144	


...


trial: 218	 th: 0.494	 resnet: 0.943	 tf: 0.005	 emb: 0.052	
FOLD 0 ROC AUC:	 0.5429155486866468	 SCORE: 3990.146335634498	 TIME: 32.6min
FOLD 1 ROC AUC:	 0.5514070239483237	 SCORE: 4107.252578636724	 TIME: 32.8min
FOLD 2 ROC AUC:	 0.544092691815084	 SCORE: 3513.6849956764104	 TIME: 32.9min
FOLD 3 ROC AUC:	 0.5377571473335855	 SCORE: 315.8569545840735	 TIME: 33.0min
FOLD 4 ROC AUC:	 0.5403406462655275	 SCORE: 3005.9538577412272	 TIME: 33.2min
model valid score: 2986.5789	auc_score: 0.5433	


[I 2021-02-22 15:50:25,640] Trial 48 finished with value: 2986.5789444545867 and parameters: {'torch_weight': 0.9428949760166256, 'tf_weight': 0.005011515418422506, 'threshold': 0.49405822675727357}. Best is trial 37 with value: 8750.009215883245.



trial: 219	 th: 0.500	 resnet: 0.044	 tf: 0.941	 emb: 0.014	
FOLD 0 ROC AUC:	 0.629005950933953	 SCORE: 9711.404383281522	 TIME: 33.3min
FOLD 1 ROC AUC:	 0.6262235605788634	 SCORE: 10218.209224416489	 TIME: 33.4min
FOLD 2 ROC AUC:	 0.6266522919579693	 SCORE: 10039.46761624666	 TIME: 33.6min
FOLD 3 ROC AUC:	 0.6234354325163988	 SCORE: 7729.663254110151	 TIME: 33.7min
FOLD 4 ROC AUC:	 0.5695044394839505	 SCORE: 5461.787163947028	 TIME: 33.9min
model valid score: 8632.1063	auc_score: 0.6150	


[I 2021-02-22 15:51:06,819] Trial 49 finished with value: 8632.10632840037 and parameters: {'torch_weight': 0.044329778593429965, 'tf_weight': 0.941452397423731, 'threshold': 0.4998872032754138}. Best is trial 37 with value: 8750.009215883245.


FrozenTrial(number=37, values=[8750.009215883245], datetime_start=datetime.datetime(2021, 2, 22, 15, 42, 31, 323943), datetime_complete=datetime.datetime(2021, 2, 22, 15, 43, 12, 880676), params={'tf_weight': 0.9644663302790105, 'threshold': 0.49398700104586946, 'torch_weight': 0.00636465508215267}, distributions={'tf_weight': UniformDistribution(high=0.9936353449178473, low=0), 'threshold': UniformDistribution(high=0.55, low=0.45), 'torch_weight': UniformDistribution(high=1, low=0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=207, state=TrialState.COMPLETE, value=None)


In [ ]:
study.best_params

{'tf_weight': 0.9644663302790105,
 'threshold': 0.49398700104586946,
 'torch_weight': 0.00636465508215267}

In [ ]:
optuna.visualization.plot_slice(study)


In [ ]:
study.optimize(objective, n_trials=20)
print(study.best_trial)
optuna.visualization.plot_contour(study)


trial: 220	 th: 0.456	 resnet: 0.005	 tf: 0.995	 emb: 0.000	
FOLD 0 ROC AUC:	 0.6296439617310405	 SCORE: 9171.372989466543	 TIME: 34.0min
FOLD 1 ROC AUC:	 0.6267444531288243	 SCORE: 9581.550042732026	 TIME: 34.1min
FOLD 2 ROC AUC:	 0.6271750163655793	 SCORE: 9189.042804187231	 TIME: 34.3min
FOLD 3 ROC AUC:	 0.6239902561040954	 SCORE: 5981.277398977985	 TIME: 34.4min
FOLD 4 ROC AUC:	 0.5695101729027183	 SCORE: 4653.749555969162	 TIME: 34.6min
model valid score: 7715.3986	auc_score: 0.6154	


[I 2021-02-22 15:51:49,125] Trial 50 finished with value: 7715.398558266589 and parameters: {'torch_weight': 0.004511168951143424, 'tf_weight': 0.9950314646591238, 'threshold': 0.45634034112074484}. Best is trial 37 with value: 8750.009215883245.



trial: 221	 th: 0.478	 resnet: 0.087	 tf: 0.888	 emb: 0.025	
FOLD 0 ROC AUC:	 0.6282282605477723	 SCORE: 9869.834061960533	 TIME: 34.7min
FOLD 1 ROC AUC:	 0.6255809270259237	 SCORE: 10218.503168095722	 TIME: 34.8min
FOLD 2 ROC AUC:	 0.6260074649752725	 SCORE: 9870.130605119293	 TIME: 35.0min
FOLD 3 ROC AUC:	 0.6227453972290645	 SCORE: 7156.587719946295	 TIME: 35.1min
FOLD 4 ROC AUC:	 0.5694679143917416	 SCORE: 5165.7923197317195	 TIME: 35.2min
model valid score: 8456.1696	auc_score: 0.6144	


[I 2021-02-22 15:52:30,468] Trial 51 finished with value: 8456.169574970712 and parameters: {'torch_weight': 0.08708101410984881, 'tf_weight': 0.8880925136859171, 'threshold': 0.4779264045532035}. Best is trial 37 with value: 8750.009215883245.



trial: 222	 th: 0.494	 resnet: 0.034	 tf: 0.739	 emb: 0.226	
FOLD 0 ROC AUC:	 0.628886031569222	 SCORE: 9862.5324459176	 TIME: 35.4min
FOLD 1 ROC AUC:	 0.6262025650651225	 SCORE: 10357.582871089438	 TIME: 35.5min
FOLD 2 ROC AUC:	 0.6267279870299061	 SCORE: 10129.864687952813	 TIME: 35.7min
FOLD 3 ROC AUC:	 0.6236626263251287	 SCORE: 7759.228935732157	 TIME: 35.8min
FOLD 4 ROC AUC:	 0.5695681116240038	 SCORE: 5475.163273613411	 TIME: 35.9min
model valid score: 8716.8744	auc_score: 0.6150	


...


trial: 238	 th: 0.510	 resnet: 0.027	 tf: 0.461	 emb: 0.511	
FOLD 0 ROC AUC:	 0.6257090462401654	 SCORE: 8095.955456524735	 TIME: 46.4min
FOLD 1 ROC AUC:	 0.6233370754614574	 SCORE: 8560.261561760577	 TIME: 46.6min
FOLD 2 ROC AUC:	 0.624461667896928	 SCORE: 8434.21587709909	 TIME: 46.7min
FOLD 3 ROC AUC:	 0.6212895311669142	 SCORE: 7009.594376479705	 TIME: 46.9min
FOLD 4 ROC AUC:	 0.5682904039128089	 SCORE: 4805.270481674241	 TIME: 47.0min
model valid score: 7381.0596	auc_score: 0.6126	


[I 2021-02-22 16:04:15,332] Trial 68 finished with value: 7381.05955070767 and parameters: {'torch_weight': 0.027410507287467197, 'tf_weight': 0.4612766327311356, 'threshold': 0.5099101548526852}. Best is trial 37 with value: 8750.009215883245.



trial: 239	 th: 0.487	 resnet: 0.108	 tf: 0.774	 emb: 0.118	
FOLD 0 ROC AUC:	 0.627593396162297	 SCORE: 9915.366769853783	 TIME: 47.1min
FOLD 1 ROC AUC:	 0.6250904434283585	 SCORE: 10385.410395883548	 TIME: 47.3min
FOLD 2 ROC AUC:	 0.6255471001211986	 SCORE: 10055.846064286743	 TIME: 47.4min
FOLD 3 ROC AUC:	 0.6223211049202	 SCORE: 7552.4809901616745	 TIME: 47.5min
FOLD 4 ROC AUC:	 0.5694818630611056	 SCORE: 5353.728863766277	 TIME: 47.7min
model valid score: 8652.5666	auc_score: 0.6140	


[I 2021-02-22 16:04:56,596] Trial 69 finished with value: 8652.566616790406 and parameters: {'torch_weight': 0.10826487152637851, 'tf_weight': 0.7741438403948844, 'threshold': 0.4868953506419893}. Best is trial 37 with value: 8750.009215883245.


FrozenTrial(number=37, values=[8750.009215883245], datetime_start=datetime.datetime(2021, 2, 22, 15, 42, 31, 323943), datetime_complete=datetime.datetime(2021, 2, 22, 15, 43, 12, 880676), params={'tf_weight': 0.9644663302790105, 'threshold': 0.49398700104586946, 'torch_weight': 0.00636465508215267}, distributions={'tf_weight': UniformDistribution(high=0.9936353449178473, low=0), 'threshold': UniformDistribution(high=0.55, low=0.45), 'torch_weight': UniformDistribution(high=1, low=0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=207, state=TrialState.COMPLETE, value=None)


In [ ]:
study.optimize(objective, n_trials=20)
print(study.best_trial)
optuna.visualization.plot_contour(study)


trial: 240	 th: 0.478	 resnet: 0.477	 tf: 0.410	 emb: 0.113	
FOLD 0 ROC AUC:	 0.612341746921792	 SCORE: 8692.617029590256	 TIME: 47.8min
FOLD 1 ROC AUC:	 0.6122907262417321	 SCORE: 9223.511182237908	 TIME: 48.0min
FOLD 2 ROC AUC:	 0.6124753840128931	 SCORE: 8761.318412848952	 TIME: 48.1min
FOLD 3 ROC AUC:	 0.6078662998118978	 SCORE: 5722.978776610042	 TIME: 48.2min
FOLD 4 ROC AUC:	 0.565912981098064	 SCORE: 4610.883884416257	 TIME: 48.4min
model valid score: 7402.2619	auc_score: 0.6022	


[I 2021-02-22 16:05:38,698] Trial 70 finished with value: 7402.261857140683 and parameters: {'torch_weight': 0.4770986680394644, 'tf_weight': 0.40959775232223644, 'threshold': 0.4783025385027267}. Best is trial 37 with value: 8750.009215883245.



trial: 241	 th: 0.490	 resnet: 0.069	 tf: 0.734	 emb: 0.197	
FOLD 0 ROC AUC:	 0.6282233866192439	 SCORE: 9929.907799387216	 TIME: 48.5min
FOLD 1 ROC AUC:	 0.6256500808826786	 SCORE: 10393.025325788638	 TIME: 48.7min
FOLD 2 ROC AUC:	 0.6261487712891374	 SCORE: 10150.30870360963	 TIME: 48.8min
FOLD 3 ROC AUC:	 0.6230247743891835	 SCORE: 7559.311170495811	 TIME: 48.9min
FOLD 4 ROC AUC:	 0.5695511699522072	 SCORE: 5437.287758788427	 TIME: 49.1min
model valid score: 8693.9682	auc_score: 0.6145	


[I 2021-02-22 16:06:20,724] Trial 71 finished with value: 8693.968151613944 and parameters: {'torch_weight': 0.06874132046797848, 'tf_weight': 0.7338964108657864, 'threshold': 0.4896473974126721}. Best is trial 37 with value: 8750.009215883245.



trial: 242	 th: 0.496	 resnet: 0.073	 tf: 0.730	 emb: 0.198	
FOLD 0 ROC AUC:	 0.6281344441595629	 SCORE: 9719.863345014874	 TIME: 49.2min
FOLD 1 ROC AUC:	 0.6255754969105515	 SCORE: 10262.146429525297	 TIME: 49.4min
FOLD 2 ROC AUC:	 0.6260745349128328	 SCORE: 10016.565340663918	 TIME: 49.5min
FOLD 3 ROC AUC:	 0.6229457007644682	 SCORE: 7661.309966353983	 TIME: 49.7min
FOLD 4 ROC AUC:	 0.5695459435826872	 SCORE: 5541.119196534522	 TIME: 49.8min
model valid score: 8640.2009	auc_score: 0.6145	


...


trial: 258	 th: 0.492	 resnet: 0.044	 tf: 0.927	 emb: 0.029	
FOLD 0 ROC AUC:	 0.6290024576859032	 SCORE: 9911.75694640231	 TIME: 60.4min
FOLD 1 ROC AUC:	 0.6262275040012022	 SCORE: 10425.559882287209	 TIME: 60.6min
FOLD 2 ROC AUC:	 0.6266597987956382	 SCORE: 10219.954265762713	 TIME: 60.7min
FOLD 3 ROC AUC:	 0.6234541340391239	 SCORE: 7664.939620340325	 TIME: 60.9min
FOLD 4 ROC AUC:	 0.5695154921455754	 SCORE: 5506.647708832198	 TIME: 61.0min
model valid score: 8745.7717	auc_score: 0.6150	


[I 2021-02-22 16:18:15,229] Trial 88 finished with value: 8745.77168472495 and parameters: {'torch_weight': 0.04428815882980704, 'tf_weight': 0.9269846453970394, 'threshold': 0.4918221155602813}. Best is trial 81 with value: 8769.290346644093.



trial: 259	 th: 0.476	 resnet: 0.021	 tf: 0.932	 emb: 0.047	
FOLD 0 ROC AUC:	 0.6293855216051779	 SCORE: 9917.749108134953	 TIME: 61.1min
FOLD 1 ROC AUC:	 0.6265529779320343	 SCORE: 10286.838491005063	 TIME: 61.3min
FOLD 2 ROC AUC:	 0.626992536548143	 SCORE: 9796.404842893242	 TIME: 61.4min
FOLD 3 ROC AUC:	 0.6238259111548088	 SCORE: 7206.071849818638	 TIME: 61.6min
FOLD 4 ROC AUC:	 0.569539759711912	 SCORE: 5192.0331358495205	 TIME: 61.7min
model valid score: 8479.8195	auc_score: 0.6153	


[I 2021-02-22 16:18:57,273] Trial 89 finished with value: 8479.819485540283 and parameters: {'torch_weight': 0.02144559583916452, 'tf_weight': 0.9317616526900653, 'threshold': 0.4761417979929181}. Best is trial 81 with value: 8769.290346644093.


FrozenTrial(number=81, values=[8769.290346644093], datetime_start=datetime.datetime(2021, 2, 22, 16, 12, 38, 843109), datetime_complete=datetime.datetime(2021, 2, 22, 16, 13, 20, 915098), params={'tf_weight': 0.9652059108090977, 'threshold': 0.48866830419666285, 'torch_weight': 0.0014678161049516518}, distributions={'tf_weight': UniformDistribution(high=0.9985321838950484, low=0), 'threshold': UniformDistribution(high=0.55, low=0.45), 'torch_weight': UniformDistribution(high=1, low=0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=251, state=TrialState.COMPLETE, value=None)


In [ ]:
study.best_params

{'tf_weight': 0.9652059108090977,
 'threshold': 0.48866830419666285,
 'torch_weight': 0.0014678161049516518}

In [ ]:
while True:
    study.optimize(objective, n_trials=20)
    print(study.best_trial)
    optuna.visualization.plot_contour(study)


In [ ]:
study.best_params